## Data Cleaning - Part I

------

### Cleaning Guide:

- [Importing Data](#loading_data)
- [Data Inspection](#inspection)
- [Realized & Estimate Price](#realized_price) 
- [Print Variables](#print_variables) 
- [Datetime Variables](#datetime_variables)
- [Dimensions Variable](#dimensions_variables)
- [Manual Cleaning by inspection](#by_inspection)

In [1]:
import pandas as pd
import numpy as np
import re
from datetime import datetime
import warnings
warnings.simplefilter('ignore')

<a id="loading_data"></a>
### Loading in Auction data:

---

- Scraped data is stored is multiple csv files
- Load/Read acquired data from web-scrape 
- Append to a list, giving a list of dataframe

In [14]:
#code for loading dataframe into a list:

listofdata = []

for i in range(1,100000, 1000):
    listofdata.append(pd.read_csv('./dataset/df_{}_{}.csv'.format(i,i+999)))

for i in range(1,10000,100):
    try:
        listofdata.append(pd.read_csv('./dataset/df_{}_{}.csv'.format(i,i+99)))
    except:
        pass

listofdata.append(pd.read_csv('./dataset/df_10001_11000_low.csv'))
listofdata.append(pd.read_csv('./dataset/df_11001_12000_low.csv')) 
listofdata.append(pd.read_csv('./dataset/df_100001_low.csv'))


#### Concatenate the list of dataframes in to one large dataframe with all the scraped data:

In [15]:
#Combining to a single dataframe:
df = pd.concat(listofdata)


In [16]:
df.shape #166,602 rows X 16 columns

(166602, 16)

In [17]:
#drop duplicates:
df.drop_duplicates(inplace = True)


In [18]:
df.shape #165,444 rows X 16 columns

(165444, 16)

### Data Inspection:
----
<a id="inspection"></a>

In [19]:
df.head()

,Art_medium,Artist,Artist_url,Dimensions,Estimate_Price,Image_url,Realized_Price,Sale_date,Signed,Similar_items,Title,Title_url,Unnamed: 0,Unnamed: 0.1,Venue,Year
0,"Color offset lithogragh, screen and pochoir wi...",Robert Rauschenberg,https://www.mutualart.com/Artist/Robert-Rausch...,27 x 21 1/2 inches,"800 - 1,120 USD",https://media.mutualart.com/Images/2012_10/12/...,"1,000 USD","Oct. 16, 2012",Signed.9/78,https://www.mutualart.com/Artwork/HORSEFEATHER...,HORSEFEATHERS THIRTEEN-XIII,https://www.mutualart.com/Artwork/HORSEFEATHER...,0,0.0,Doyle New York,1972
1,color soap ground aquatint,Pat Steir,https://www.mutualart.com/Artist/Pat-Steir/84D...,"27.75"" x 25.25"" in",NaN,https://media.mutualart.com/Images/2012_10/21/...,"1,000 USD","Oct. 24, 2012",Signed.12/40,https://www.mutualart.com/Artwork/Starry-Night...,Starry Night June,https://www.mutualart.com/Artwork/Starry-Night...,1,1.0,Dallas Auction Gallery,1996
2,"Color etching with aquatint, drypoint & hand-c...",Louis Icart,https://www.mutualart.com/Artist/Louis-Icart/9...,27 1/2 x 18 3/4 inches,NaN,https://media.mutualart.com/Images/2013_03/13/...,"1,000 USD","May 08, 2002",Signed.,https://www.mutualart.com/Artwork/Orchids/0BA9...,Orchids,https://www.mutualart.com/Artwork/Orchids/0BA9...,2,2.0,Freeman's,1937
3,engraved,John Bogle,https://www.mutualart.com/Artist/John-Bogle/1B...,51mm (2in) high,NaN,https://media.mutualart.com/Images/2012_11/21/...,"=""1,000 USD","Nov. 28, 2012",Signed.,https://www.mutualart.com/Artwork/Mrs-Anne-Mur...,Mrs Anne Murray (née Campbell-Colquhoun) (b. c...,https://www.mutualart.com/Artwork/Mrs-Anne-Mur...,3,3.0,Bonhams Oxford,1794
4,"Color lithograph, screenprint, pochoir, collag...",Robert Rauschenberg,https://www.mutualart.com/Artist/Robert-Rausch...,21 1/2x13 in,NaN,https://media.mutualart.com/Images/2012_11/14/...,"1,000 USD","Nov. 15, 2012",Signed.Artist'sproof,https://www.mutualart.com/Artwork/Horsefeather...,Horsefeathers Thirteen XV,https://www.mutualart.com/Artwork/Horsefeather...,4,4.0,Swann Auction Galleries,1972


In [20]:
df.isnull().sum()

Art_medium         14818
Artist               369
Artist_url           369
Dimensions         10834
Estimate_Price     92151
Image_url            165
Realized_Price         6
Sale_date            169
Signed                 1
Similar_items        207
Title                  0
Title_url              0
Unnamed: 0             0
Unnamed: 0.1      163839
Venue                169
Year               29771
dtype: int64

In [9]:
df.drop(columns = ['Unnamed: 0','Unnamed: 0.1'], inplace = True)

In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 165444 entries, 0 to 3051
Data columns (total 14 columns):
Art_medium        150626 non-null object
Artist            165075 non-null object
Artist_url        165075 non-null object
Dimensions        154610 non-null object
Estimate_Price    73293 non-null object
Image_url         165279 non-null object
Realized_Price    165438 non-null object
Sale_date         165275 non-null object
Signed            165443 non-null object
Similar_items     165237 non-null object
Title             165444 non-null object
Title_url         165444 non-null object
Venue             165275 non-null object
Year              135673 non-null object
dtypes: object(14)
memory usage: 18.9+ MB


<b> Variables are all object type, the following need to be converted:</b>

- Estimate Price (int)
- Realized Price (int)
- Sale_date (datetime)
- Signed (binary)
- Year (int)
- Dimension (float)

<a id="realized_price"></a>
### Realized & Estimate Price:

---

- Since <u>Realized Price is our target variable</u>, we can drop all <u>NaN values</u> in this column before further inspection/cleaning: 

In [11]:
df.dropna(subset = ['Realized_Price'], inplace = True)

#### Converting Realized Price to int type:

In [12]:
df.Realized_Price = df.Realized_Price.apply(lambda x: (int(x.replace(',','').replace('=','').replace('"','').split(' ')[0])))

#### Converting Estimated Price to int type:

In [13]:
def estimate_clean(x):
    try:
        list_p = x.replace('USD','').replace(' ','').replace(',','').split('-')
        list_n = []
        for i in list_p:
            list_n.append(int(i))
        price_e = sum(list_n)/len(list_n)
        return price_e
    except:
        return x

In [14]:
df.Estimate_Price = df.Estimate_Price.apply(estimate_clean)

<a id="print_variables"></a>
### Print Variables:

---

- Convert Signed variable to binary, 1 for Signed, 0 for Not Signed.
- Signed Variable contains additional information about the number of prints the artwork was part of e.g. 10/100 (#10 out of 100)
- As the number of a print may be important  
- Assigning new column:  the <u>total number of prints</u> of the artwork

In [21]:
#Signed column currently looks like:
df.Signed[:5]

0             Signed.9/78
1            Signed.12/40
2                 Signed.
3                 Signed.
4    Signed.Artist'sproof
Name: Signed, dtype: object

#### Seperating and organising relevant information:

In [23]:
#if nothing but 'Signed.' is said in the cell, then the artwork is assumed to be a unique print
#Otherwise the print number information is seperated out into a list for further analysis

def print_num(x):
    if x != 'Signed.':
        try:
            return x.split('.')[1]
        except:
            return x
    else:
        return 1

In [24]:
df['Print_number'] = df.Signed.apply(print_num)

In [27]:
df.Print_number[:5]

0             9/78
1            12/40
2                1
3                1
4    Artist'sproof
Name: Print_number, dtype: object

In [17]:
#Fill in NaN cells with 'No' as it is Not signed.

df.Signed.fillna('No', inplace = True)
df.Print_number.fillna('No', inplace = True)

In [18]:
df.Print_number.isnull().sum()

0

Find and extract the number of prints

<b>Problem</b>
1. Column contains roman numerals (i.e I,IX,XXX) as well as latin numbers (i.e. 1,100) 
    - Hence will need to converting roman numberals to latin numbers
- Need to extract total number of prints amongst other numbers

**Solution:** ... Regex!


Function for identifying the max latin number:

In [20]:
def total_print(x):
    k =[]

    try:
        t = re.findall(r"\d+", x)        
        for i in t:
            k.append(int(i))
        tot = max(k)
    except:
        tot = x
        
    return tot

Function for converting roman numerals to latin numbers:

In [ ]:
def roman_to_int(s):
        rom_val = {'I': 1, 'V': 5, 'X': 10, 'L': 50, 'C': 100, 'D': 500, 'M': 1000}
        int_val = 0
        for i in range(len(s)):
            if i > 0 and rom_val[s[i]] > rom_val[s[i - 1]]:
                int_val += rom_val[s[i]] - 2 * rom_val[s[i - 1]]
            else:
                int_val += rom_val[s[i]]
        return int_val

Function for identifying the max roman numeral:

In [21]:
def roman_print(x):
    k=[]
    if isinstance(x,str):
        try:
            rl = re.findall(r'[I,V,X,L,C,D,M]+',x)
            for l in rl:
                k.append(roman_to_int(l))
            tot = max(k)
        except:
            tot = x
    else:
        tot = x
    return tot

In [22]:
#manual cleaning of words in the data column.
def word_print(x):
    exceptionlist = ["Artist'sproof","artist'sproof",'No',"Arist'sProof",
                     "Artist'sProof","Artist's","Artist'sproofs","Artist'sproff",
                     'ArtistPrint','ArtistProof','ArtistProofsfromaneditionofsmalledition','Artistproof',
                     "Artists'proof","Artists'proofs",'ArtistsProof','Artistsproof','Artist’sproof',
                     "EachArtist'sproofs","Eachartist'sproof","Eachprinter'sproof","Epreuved'Artiste",
                     "Epreuved'artiste",'FirstParis','Firstedition,IkedaPublishers','H','Horscommerce',
                     'Horsdecommerce','Horsdecommerce,','Onefromanunknowneditionsize','Onefromanunlimitededition',
                     'Oneofapprox','Oneofjustafewtrialproofs','Oneofonlyseveralproofs',
                     'Oneofonlyseveralproofs(therewasnopublishededition)','Oneofsevenrecordedproofs',
                     'Oneofthefewfirststateproofs', "Printer'sProof","Printer'sproof",'PrintersProof','Proof',
                     'Rare','Soleknowncopyfromanedition',"Thisworkisuniqueplusoneartist'sproof","Trial'sproof",
                     'TrialProof','TrialProofNo','Trialproof','TrialproofbeforethePANedition','Uniquefromanedition',
                     'Uniquepiecefromanedition','ararebonàtirerimpression','arareearlyproof',"artisrt'sproof",
                     "artist'sproof',artist'sproofs","artist'sproof,arareprint,fromonlyafewproofs,theprintwasnevereditioned",
                     'auniquecolorvariant','auniquetrialproofofthisveryrareprint','beforetheeditionofonehundred',
                     'bonatirer','bonàtirer','bonàtirerproof','one/one',"printer'sproof",
                     "printer'sproof,artist'sproof","thisworkisanartist'sproof",'thisworkisfromalimitededition',
                     'trailproof','trialproof','unique','uniqueartwork','uniquefromanedition',"withartist'sproof",
                     "withartist'sproofs","withprinter'sproof",'workingcolorproof','workingproof',"épreuved'artist",
                     "épreuved'artiste"]
    if isinstance(x,int):
        return x
    elif x in exceptionlist:
        tot = 1
    else:
        tot = np.nan
    
    return tot
    

In [23]:
df['Total_prints'] = df.Print_number.apply(total_print)

In [24]:
df['Total_prints'] = df.Total_prints.apply(roman_print)

In [25]:
df['Total_prints'] = df.Total_prints.apply(word_print)

In [26]:
df.Total_prints.isnull().sum()

2493

#### Signed or Not:

- Setting variable signed to being binary (1 = Signed, 0 = Not Signed)

In [27]:
def binary_signed(x):
    try:
        if x == 'No':
            return 0
        else:
            return 1
    except:
        return 'error'

In [28]:
df.Signed = df.Signed.apply(binary_signed)

In [29]:
df.Signed.value_counts()

1    125768
0     39670
Name: Signed, dtype: int64

<a id="datetime_variables"></a>
### Datetime Variables:

---

- Converting dates to datetime type (i.e. Sale_date and Year):

In [28]:
df.Sale_date[:5]

0    Oct. 16, 2012
1    Oct. 24, 2012
2     May 08, 2002
3    Nov. 28, 2012
4    Nov. 15, 2012
Name: Sale_date, dtype: object

In [29]:
def convert_datetime(x):
    try:
        conv = datetime(x)
    except:
        try:
            conv = x.replace(',','').replace('.','')
#             conv = datetime.strptime(symbolsrm, '%b %d %Y')
        except:
            conv = np.nan
    
    return conv

In [30]:
df.Sale_date = df.Sale_date.apply(convert_datetime)

<u>Taking a look at the Year column:</u>
- Some are in century format
- Some are combined with words

In [34]:
df.Year[28:36]

28             NaN
29    19th Century
30             NaN
31            1893
32            1836
33            1918
34      circa 1890
35      circa 1950
Name: Year, dtype: object

In [36]:
def year_clean(x):
    try:
        t = re.findall(r"\d+", x)
        if len(t)==1:
            if len(t[0])==4 and int(t[0])<2020:
                w = t[0]
            elif len(t[0])==2 and int(t[0])<22:
                w = t[0]+'00'
            else:
                w = np.nan
            return datetime.strptime(w, '%Y').year
        
        elif len(t)==2:
            if len(t[0])==4 and int(t[0])<2020:
                w = t[0]
            elif len(t[0])==2 and int(t[0])<22:
                w = t[0]+'00'
            elif len(t[1])==4 and int(t[1])<2020:
                w = t[1]
            elif len(t[1])==2 and int(t[1])<22:
                w = t[1]+'00'
            else:
                w = np.nan
            return datetime.strptime(w, '%Y').year
        
        elif len(t)>2:
            if len(t[0])==4 and int(t[0])<2020:
                w = t[0]
            elif len(t[0])==2 and int(t[0])<22:
                w = t[0]+'00'
            elif len(t[1])==4 and int(t[1])<2020:
                w = t[1]
            elif len(t[1])==2 and int(t[1])<22:
                w = t[1]+'00'
            elif len(t[2])==4 and int(t[2])<2020:
                w = t[2]
            elif len(t[2])==2 and int(t[2])<22:
                w = t[1]+'00'
            else:
                w = np.nan
            return datetime.strptime(w, '%Y').year
        
        else:
            return np.nan
    except:
        return np.nan

In [37]:
df.Year = df.Year.apply(year_clean)

In [38]:
df.Year[28:36]

28       NaN
29    1900.0
30       NaN
31    1893.0
32    1836.0
33    1918.0
34    1890.0
35    1950.0
Name: Year, dtype: float64

<a id="dimensions_variables"></a>
### Dimensions Variable:

---

<b>Problem:</b>
1. There are multiple units in the column, cm, mm, inches and ft
2. One cell may contain more than one unit

<b>Solution:</b>
<u>Using RegEx:</u> 
- find the unit(s) used
- find the length of dimensions


In [35]:
def find_units(x):
    try:
        unit = []
        p = re.findall(r'(?:cm|mm|in|ft|")',x)
        r = ['in' if i =='"' else i for i in p]
        
        for i in r:
            if i not in unit:
                unit.append(i)
            else:
                pass
        return unit
    except:
        return x

In [36]:
df['Dimension_units'] = df.Dimensions.apply(find_units)

In [37]:
df.Dimension_units.value_counts()

[in]            41080
[cm]            32348
[mm]            19975
[mm, in]        19090
[in, mm]        18538
[in, cm]        12893
[cm, in]         7898
[]               2743
[mm, ft]            5
[ft, cm]            4
[in, cm, mm]        4
[mm, in, ft]        4
[cm, mm]            3
[in, cm, ft]        2
[mm, cm]            2
[cm, mm, in]        2
[ft, mm]            2
[mm, ft, in]        2
[in, ft, mm]        2
[cm, ft]            1
[in, ft, cm]        1
[mm, in, cm]        1
[ft, in, cm]        1
[cm, in, mm]        1
[ft, mm, in]        1
[cm, ft, in]        1
[ft, in]            1
Name: Dimension_units, dtype: int64

In [38]:
def dimension_value(x):
    try:
        r = re.findall(r'[\W,\d]+',x)
        d = []
        for i in r:
            d.append(i.strip().replace('"','').replace('-',' ').replace('(', ' ').replace(')', ' '))
        e = []
        for j in d:
            e.append(j.split())
        f = []
        for k in e:
            k3 =[]
            for k1 in k:
                if '/'in k1:
                    k2 = k1.split('/')
                    k3.append(int(k2[0])/int(k2[1]))
                else:
                    try:
                        k3.append(float(k1))
                    except:
                        pass
            f.append(sum(k3))
        g =[]
        for g1 in f:
            if g1 > 0:
                g.append(g1)
            else:
                pass
        return g
    except:
        return x
    

In [39]:
df['Dimemsion_values_d'] = df.Dimensions.apply(dimension_value)

In [40]:
df.reset_index(drop=True, inplace = True)

In [41]:
df.shape[0]

165438

<u>Convert all dimensions to the same unit: inches</u> 
- Find position of dimensions in inches
- If no dimensions in inches then convert to inches accordingly

In [42]:
unit_conv_inch = []

for i in range(df.shape[0]):
    a = df.Dimension_units[i]
    b = df.Dimemsion_values_d[i]
    
    try:
        if len(a) == 1:
            if len(b)==1 or len(b)==2:
                if 'in' in a:
                    unit_conv_inch.append(b)
                elif 'cm' in a:
                    b1 = []
                    for ii in b:
                        b1.append(ii*0.394)
                    unit_conv_inch.append(b1)
                elif 'mm' in a:
                    b1_ = []
                    for ii_ in b:
                        b1_.append(ii_*0.0394)
                    unit_conv_inch.append(b1_)
                elif 'ft' in a:
                    _b1 = []
                    for _ii in b:
                        _b1.append(_ii*12)
                    unit_conv_inch.append(_b1)
                else:
                    unit_conv_inch.append(np.nan)
            elif len(b)==4:
                if 'in' in a:
                    unit_conv_inch.append([b[0],b[1]])
                elif 'cm' in a:
                    b1 = []
                    for ii in b:
                        b1.append(ii*0.394)
                    unit_conv_inch.append([b1[0],b1[1]])
                elif 'mm' in a:
                    b1_ = []
                    for ii_ in b:
                        b1_.append(ii_*0.0394)
                    unit_conv_inch.append([b1_[0],b1_[1]])
                elif 'ft' in a:
                    _b1 = []
                    for _ii in b:
                        _b1.append(_ii*12)
                    unit_conv_inch.append([_b1[0],_b1[1]])
                else:
                    unit_conv_inch.append(np.nan)
            else:
                unit_conv_inch.append(np.nan)
        elif len(a) == 2:
            if 'in' in a and len(b)==2:
                ind = a.index('in')
                unit_conv_inch.append([b[ind]])
            elif 'mm' in a and len(b)==2:
                ind_ = a.index('mm')
                unit_conv_inch.append([b[ind_]*0.0394])
            elif 'cm' in a and len(b)==2:
                _ind = a.index('cm')
                unit_conv_inch.append([b[_ind]*0.394])

            elif 'in' in a and len(b)>3:
                inde = a.index('in')
                unit_conv_inch.append([b[inde+inde],b[inde+inde+1]])
            elif 'mm' in a and len(b)>3:
                inde_ = a.index('mm')
                unit_conv_inch.append([b[inde_+inde_]*0.0394,b[inde_+inde_+1]*0.0394])
            elif 'cm' in a and len(b)>3:
                _inde = a.index('cm')
                unit_conv_inch.append([b[_inde+_inde]*0.394,b[_inde+_inde+1]*0.394])
            else:
                unit_conv_inch.append(np.nan)

        elif len(a) == 3:
            if 'in' in a and len(b)==3:
                indee = a.index('in')
                unit_conv_inch.append([b[indee]])
            elif 'mm' in a and len(b)==3:
                indee_ = a.index('mm')
                unit_conv_inch.append([b[indee_]*0.0394])
            elif 'cm' in a and len(b)==3:
                _ind = a.index('cm')
                unit_conv_inch.append([b[_ind]*0.394])

            elif 'in' in a and len(b)>5:
                indeee = a.index('in')
                unit_conv_inch.append([b[indeee+indeee],b[indeee+indeee+1]])
            elif 'mm' in a and len(b)>3:
                indeee_ = a.index('mm')
                unit_conv_inch.append([b[indeee_+indeee_]*0.0394,b[indeee_+indeee_+1]*0.0394])
            elif 'cm' in a and len(b)>3:
                _indeee = a.index('cm')
                unit_conv_inch.append([b[_indeee+_indeee]*0.394,b[_indeee+_indeee+1]*0.394])
            else:
                unit_conv_inch.append(np.nan)
        else:
            unit_conv_inch.append(np.nan)
    
    except:
        unit_conv_inch.append(np.nan)
            
            
        

In [43]:
df['dimension_inch'] = unit_conv_inch

In [44]:
lengthdimen =[]
for i in range(df.shape[0]):
    try:
        j = len(df.dimension_inch[i])
        if j not in lengthdimen:
            lengthdimen.append(j)
    except:
        pass

In [45]:
lengthdimen

[2, 1]

<b> Converting dimensions given in lenght to a single Area variable, inches**2 </b>
- Find the area of the artwork by multiplying the dimensions in inches together.
- This gives a single number for describing the artwork dimension allowing for easier analysis.
- For data with two variable multiply by each other and for ones with one variable, assume shape of artwork to be square so multiply by itself.


In [46]:
def dimension_area(x):
    try:
        if len(x)==2:
            return x[0]*x[1]
        elif len(x) == 1:
            return x[0]*x[0]
        else:
            return np.nan
    except:
        return np.nan
    

In [47]:
df['Area_dimension'] = df.dimension_inch.apply(dimension_area)

### Saving dataframe to .csv file
---

In [69]:
df.to_csv('auction_data.csv')